In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Wikipedia Question Answering

This notebook walks through using Riva NLP Services to generate an answer to an incoming user question using Riva' Question and Answering modul by querying Wikipedia for a summary of the topic in question.

First make sure the Wikipedia API is installed. Alter the bellow command if you use a different package manager. Then we import the required packages.



In [ ]:
!pip install wikipedia

In [3]:
import wikipedia as wiki

import grpc

import riva_api.riva_nlp_pb2 as rnlp
import riva_api.riva_nlp_pb2_grpc as rnlp_srv

### Wikipedia Summary
Next we accept user input. We use the Wikipedia API to find the most relevant articles. For the purpose of this example, we combine the top few article summaries. You can change the number of articles with `max_articles_combine`.

In [4]:
input_query = "What is speech recognition?"

wiki_articles = wiki.search(input_query)
max_articles_combine = 3
combined_summary = ""

if len(wiki_articles) == 0:
    print("ERROR: Could not find any matching results in Wikipedia.")
else:
    for article in wiki_articles[:min(len(wiki_articles), max_articles_combine)]:
        print(f"Getting summary for: {article}")
        combined_summary += "\n" + wiki.summary(article)

Getting summary for: Speech recognition
Getting summary for: Windows Speech Recognition
Getting summary for: Speaker recognition


### Query Riva Server

Lastly, we define the GRPC channel to send a request to the Riva server, and define a corresponding request. This allows us to query the Riva server with the given input query and the context taken from Wikipedia.

Make sure you set your grpc channel to the appropriate port.

In [5]:
# Alter if using a different port
channel = grpc.insecure_channel('localhost:50051')

riva_nlp = rnlp_srv.RivaLanguageUnderstandingStub(channel)

req = rnlp.NaturalQueryRequest()

req.query = input_query
req.context = combined_summary
resp = riva_nlp.NaturalQuery(req)

print(f"Query: {input_query}")
print(f"Answer: {resp.results[0].answer}")

Query: What is speech recognition?
Answer: an interdisciplinary subfield of computer science and computational linguistics
